# 1. Импортирование библиотек и парсинг данных

In [64]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
import torch
import re
import os
from glob import glob
from torch.utils.data import TensorDataset, DataLoader
from torchvision import models, transforms
from tqdm import tqdm
# Отключаем проверку
import warnings
warnings.filterwarnings("ignore")

In [105]:
list_bmp = []
list_frame = []
def parsing_bmp(dir):
    list = os.listdir(dir)    
    for item in glob(dir+'\\'+'*.bmp'):
        list_bmp.append(dir+'\\'+item)
    for item in glob(dir+'\\'+'*.frame'):
        list_frame.append(dir+'\\'+item)
    for dir_for in list:
        if os.path.isdir(dir+'\\'+dir_for):
            print(dir_for)
            parsing_bmp(dir+'\\'+dir_for)
parsing_bmp('Трубы\\FRAMES')

0
1530
1538
1538mov
2070
2070_1
410
414
cl
cl2
d
miss
mv
mv_1
n
n1
nn
p
unk
грязь
лапа
мелочь
потертость
труба
2020.10.15
труба 1
труба 2
труба 3
труба 4
труба 5
труба 6
труба 7
труба 8
2020.10.16
Труба 1
Труба 10
Труба 2
Труба 3
Труба 4
Труба 5
Труба 6
Труба 7
Труба 8
Труба 9
2020.10.17
Труба 1
Труба 2
Труба 3
Труба 4
Труба 5
Труба 6
Труба 7
Труба 8
2020.10.20
2020.10.21
2020.10.23
2020.10.24
Труба 4
Труба 5
Труба 6
Труба 7
2020.10.25
2020.10.26
2021.05.27
2021.05.28
2021.08.30
2
3
4
5
2021.08.31
1
2021.09.01
01
02
03
04
05
06
07
08
09
1
10
2021.09.02
01
02
03
мв
04
05
06
07
08
09
10
2021.09.03
01
mnp
обр накол
02
03
mnp
04
05
06
08
09
10
2021.09.23
2021.10.24
2022.05.24
0
1
2
3
2022.06.21
0
1
2022.06.23
0
2022.06.27
0
2022.07.21
0
1
2022.08.02
0
1
2
2022.08.17
0
2022.08.23
0
2022.09.05
0
2022.10.03
2022.10.08
2022.10.17
2022.10.24
2022.10.27
2022.10.31
2022.11.22
2022.11.24
2022.11.25
0
2022.12.06
2022.12.07
1
2022.12.08
2022.12.13
2022.12.15
0
1
2022.12.27
0
1
2023.01.07
0
2023.01.0

In [107]:
list_bmp

['Трубы\\FRAMES\\Трубы\\FRAMES\\frame0000.bmp',
 'Трубы\\FRAMES\\0\\1530\\Трубы\\FRAMES\\0\\1530\\frame0000.bmp',
 'Трубы\\FRAMES\\0\\1530\\Трубы\\FRAMES\\0\\1530\\frame0001.bmp',
 'Трубы\\FRAMES\\0\\1530\\Трубы\\FRAMES\\0\\1530\\frame0002.bmp',
 'Трубы\\FRAMES\\0\\1530\\Трубы\\FRAMES\\0\\1530\\frame0003.bmp',
 'Трубы\\FRAMES\\0\\1530\\Трубы\\FRAMES\\0\\1530\\frame0004.bmp',
 'Трубы\\FRAMES\\0\\1530\\Трубы\\FRAMES\\0\\1530\\frame0005.bmp',
 'Трубы\\FRAMES\\0\\1530\\Трубы\\FRAMES\\0\\1530\\frame0006.bmp',
 'Трубы\\FRAMES\\0\\1530\\Трубы\\FRAMES\\0\\1530\\frame0007.bmp',
 'Трубы\\FRAMES\\0\\1530\\Трубы\\FRAMES\\0\\1530\\frame0008.bmp',
 'Трубы\\FRAMES\\0\\1530\\Трубы\\FRAMES\\0\\1530\\frame0009.bmp',
 'Трубы\\FRAMES\\0\\1530\\Трубы\\FRAMES\\0\\1530\\frame0010.bmp',
 'Трубы\\FRAMES\\0\\1530\\Трубы\\FRAMES\\0\\1530\\frame0011.bmp',
 'Трубы\\FRAMES\\0\\1530\\Трубы\\FRAMES\\0\\1530\\frame0012.bmp',
 'Трубы\\FRAMES\\0\\1538\\Трубы\\FRAMES\\0\\1538\\frame0000.bmp',
 'Трубы\\FRAMES\\0\\1538\\Тр

In [108]:
list_frame

['Трубы\\FRAMES\\Трубы\\FRAMES\\frame0000.frame',
 'Трубы\\FRAMES\\0\\1530\\Трубы\\FRAMES\\0\\1530\\frame0000.frame',
 'Трубы\\FRAMES\\0\\1530\\Трубы\\FRAMES\\0\\1530\\frame0001.frame',
 'Трубы\\FRAMES\\0\\1530\\Трубы\\FRAMES\\0\\1530\\frame0002.frame',
 'Трубы\\FRAMES\\0\\1530\\Трубы\\FRAMES\\0\\1530\\frame0003.frame',
 'Трубы\\FRAMES\\0\\1530\\Трубы\\FRAMES\\0\\1530\\frame0004.frame',
 'Трубы\\FRAMES\\0\\1530\\Трубы\\FRAMES\\0\\1530\\frame0005.frame',
 'Трубы\\FRAMES\\0\\1530\\Трубы\\FRAMES\\0\\1530\\frame0006.frame',
 'Трубы\\FRAMES\\0\\1530\\Трубы\\FRAMES\\0\\1530\\frame0007.frame',
 'Трубы\\FRAMES\\0\\1530\\Трубы\\FRAMES\\0\\1530\\frame0008.frame',
 'Трубы\\FRAMES\\0\\1530\\Трубы\\FRAMES\\0\\1530\\frame0009.frame',
 'Трубы\\FRAMES\\0\\1530\\Трубы\\FRAMES\\0\\1530\\frame0010.frame',
 'Трубы\\FRAMES\\0\\1530\\Трубы\\FRAMES\\0\\1530\\frame0011.frame',
 'Трубы\\FRAMES\\0\\1530\\Трубы\\FRAMES\\0\\1530\\frame0012.frame',
 'Трубы\\FRAMES\\0\\1538\\Трубы\\FRAMES\\0\\1538\\frame0000.frame'

In [111]:
print(f'Файлов формата bmp {len(list_bmp)} и формата frame {len(list_frame)}')

Файлов формата bmp 7808 и формата frame 7808


# 2. Чтение изображения